# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [1]:
import pandas as pd
import sys
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from tqdm import tqdm
from sklearn import svm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, StratifiedKFold, GridSearchCV
import joblib


## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore).
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [2]:
df = pd.read_csv("../data/dayofweek-not-scaled.csv")
df

,numTrials,hour,dayofweek,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,...,labname_lab02,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1
0,1,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,9,20,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1682,6,20,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1683,7,20,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1684,8,20,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('dayofweek', axis=1), df['dayofweek'], test_size=0.2, random_state=21, stratify=df['dayofweek'])

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [4]:
my_svm = svm.SVC(random_state = 21, probability=True)

In [5]:
grid_svm = GridSearchCV(my_svm, param_grid={'kernel': ['linear', 'rbf', 'sigmoid'], 'C': [0.01, 0.1, 1, 1.5, 5, 10], \
                                            'gamma': ['scale', 'auto'], 'class_weight': ['balanced', None]}, scoring='accuracy')

In [6]:
grid_svm.fit(X_train, y_train)

GridSearchCV(estimator=SVC(probability=True, random_state=21),
             param_grid={'C': [0.01, 0.1, 1, 1.5, 5, 10],
                         'class_weight': ['balanced', None],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'sigmoid']},
             scoring='accuracy')

In [7]:
res_grid_svm = pd.DataFrame(grid_svm.cv_results_).sort_values(by='rank_test_score', ignore_index=True)
res_grid_svm

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.495701,0.013396,0.038723,0.003111,10,None,auto,rbf,"{'C': 10, 'class_weight': None, 'gamma': 'auto...",0.900000,0.848148,0.885185,0.884758,0.862454,0.876109,0.018419,1
1,0.492532,0.011658,0.040629,0.002728,10,balanced,auto,rbf,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.877778,0.851852,0.862963,0.873606,0.851301,0.863500,0.010870,2
2,0.481907,0.017510,0.043640,0.002298,5,None,auto,rbf,"{'C': 5, 'class_weight': None, 'gamma': 'auto'...",0.825926,0.811111,0.818519,0.821561,0.802974,0.816018,0.008116,3
3,0.528373,0.025249,0.046447,0.005090,5,balanced,auto,rbf,"{'C': 5, 'class_weight': 'balanced', 'gamma': ...",0.844444,0.785185,0.792593,0.817844,0.802974,0.808608,0.021007,4
4,30.392559,2.762812,0.008630,0.000342,10,balanced,auto,linear,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.729630,0.700000,0.755556,0.754647,0.665428,0.721052,0.034438,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.562804,0.019497,0.018564,0.001890,5,balanced,auto,sigmoid,"{'C': 5, 'class_weight': 'balanced', 'gamma': ...",0.144444,0.148148,0.137037,0.126394,0.092937,0.129792,0.019869,68
68,0.513817,0.019267,0.018035,0.000616,10,balanced,auto,sigmoid,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.122222,0.140741,0.129630,0.100372,0.085502,0.115693,0.020052,69
69,0.614519,0.017698,0.018585,0.001707,1.5,balanced,auto,sigmoid,"{'C': 1.5, 'class_weight': 'balanced', 'gamma'...",0.066667,0.085185,0.081481,0.078067,0.085502,0.079380,0.006913,70
70,0.627954,0.023640,0.018067,0.000492,0.1,balanced,auto,sigmoid,"{'C': 0.1, 'class_weight': 'balanced', 'gamma'...",0.062963,0.066667,0.062963,0.059480,0.059480,0.062310,0.002678,71


## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [8]:
dtc_model = DecisionTreeClassifier(random_state = 21)
grid_dtc = GridSearchCV(dtc_model, param_grid={'max_depth': [x for x in range(1,50)], \
                                               'class_weight': ['balanced', None], 'criterion': ['entropy', 'gini']}, scoring='accuracy', n_jobs=-1)
grid_dtc.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=21), n_jobs=-1,
             param_grid={'class_weight': ['balanced', None],
                         'criterion': ['entropy', 'gini'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                       23, 24, 25, 26, 27, 28, 29, 30, ...]},
             scoring='accuracy')

In [9]:
accuracy_score(y_test, grid_dtc.predict(X_test))

0.8905325443786982

In [10]:
res_grid_dtc = pd.DataFrame(grid_dtc.cv_results_).sort_values(by='rank_test_score', ignore_index=True)
res_grid_dtc

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008527,0.000298,0.002659,0.000055,balanced,gini,22,"{'class_weight': 'balanced', 'criterion': 'gin...",0.885185,0.862963,0.903704,0.881041,0.832714,0.873121,0.023998,1
1,0.008885,0.002050,0.002917,0.000210,balanced,gini,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.859259,0.903704,0.884758,0.828996,0.873121,0.026300,2
2,0.008560,0.000522,0.003126,0.000879,balanced,gini,49,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
3,0.010195,0.001385,0.002924,0.000597,balanced,gini,47,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
4,0.009425,0.001634,0.002778,0.000068,balanced,gini,46,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,0.005874,0.000175,0.002693,0.000155,balanced,gini,3,"{'class_weight': 'balanced', 'criterion': 'gin...",0.388889,0.303704,0.403704,0.427509,0.345725,0.373906,0.044064,192
192,0.005591,0.001402,0.002806,0.000114,None,gini,1,"{'class_weight': None, 'criterion': 'gini', 'm...",0.370370,0.351852,0.359259,0.353160,0.342007,0.355330,0.009338,193
193,0.004799,0.000469,0.002737,0.000145,None,entropy,1,"{'class_weight': None, 'criterion': 'entropy',...",0.370370,0.351852,0.359259,0.353160,0.342007,0.355330,0.009338,193
194,0.005260,0.000108,0.002787,0.000123,balanced,gini,1,"{'class_weight': 'balanced', 'criterion': 'gin...",0.262963,0.318519,0.266667,0.323420,0.260223,0.286358,0.028376,195


## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [11]:
rf = RandomForestClassifier(random_state = 21)
grid_rf = GridSearchCV(rf, param_grid={'max_depth': [x for x in range(1,50)], 'n_estimators': [5, 10, 50, 100], \
                                               'class_weight': ['balanced', None], 'criterion': ['entropy', 'gini']}, scoring='accuracy', n_jobs=-1)
grid_rf.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=21), n_jobs=-1,
             param_grid={'class_weight': ['balanced', None],
                         'criterion': ['entropy', 'gini'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                       23, 24, 25, 26, 27, 28, 29, 30, ...],
                         'n_estimators': [5, 10, 50, 100]},
             scoring='accuracy')

In [12]:
res_grid_rf = pd.DataFrame(grid_rf.cv_results_).sort_values(by='rank_test_score', ignore_index=True)
res_grid_rf

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.171791,0.003954,0.015398,0.001143,None,gini,28,50,"{'class_weight': None, 'criterion': 'gini', 'm...",0.922222,0.900000,0.907407,0.903346,0.888476,0.904290,0.010961,1
1,0.343410,0.013457,0.026449,0.001918,None,gini,31,100,"{'class_weight': None, 'criterion': 'gini', 'm...",0.918519,0.911111,0.900000,0.910781,0.877323,0.903547,0.014380,2
2,0.162707,0.002403,0.013865,0.000324,balanced,gini,30,50,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.881481,0.907063,0.895911,0.902817,0.013554,3
3,0.162898,0.003973,0.014016,0.000587,balanced,gini,34,50,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.892593,0.907063,0.884758,0.902809,0.013010,4
4,0.170795,0.008085,0.014929,0.001064,None,gini,29,50,"{'class_weight': None, 'criterion': 'gini', 'm...",0.918519,0.896296,0.911111,0.903346,0.884758,0.902806,0.011698,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,0.016601,0.000475,0.004311,0.000448,None,entropy,1,5,"{'class_weight': None, 'criterion': 'entropy',...",0.355556,0.366667,0.374074,0.345725,0.327138,0.353832,0.016467,780
780,0.013635,0.000553,0.003298,0.000158,balanced,entropy,2,5,"{'class_weight': 'balanced', 'criterion': 'ent...",0.318519,0.366667,0.381481,0.353160,0.345725,0.353110,0.021165,781
781,0.016993,0.001263,0.004143,0.000249,balanced,gini,2,5,"{'class_weight': 'balanced', 'criterion': 'gin...",0.311111,0.377778,0.377778,0.353160,0.312268,0.346419,0.029749,782
782,0.015600,0.000332,0.003761,0.000131,balanced,gini,1,5,"{'class_weight': 'balanced', 'criterion': 'gin...",0.262963,0.292593,0.285185,0.282528,0.293680,0.283390,0.011062,783


## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [13]:
def manual_grid_search(params):
    max_ac = 0
    min_std = 1
    res = {'n_estimators': []
         , 'max_depth': []
         , 'class_weight': []
         , 'criterion': []
         , 'mean_accuracy': []
         , 'std_accuracy': []}
    
    for val_depth in tqdm(params['max_depth'], file=sys.stdout):
        for val_est in params['n_estimators']:
            for val_claswe in params['class_weight']:
                for val_crit in params['criterion']:
                    model = RandomForestClassifier(random_state = 21, n_estimators=val_est, max_depth=val_depth, class_weight=val_claswe, criterion=val_crit)
                    cvs = cross_val_score(model, X_train, y_train, cv=5, n_jobs=-1)
                    res['n_estimators'].append(val_est)
                    res['max_depth'].append(val_depth)
                    res['class_weight'].append(val_claswe)
                    res['criterion'].append(val_crit)
                    res['mean_accuracy'].append(cvs.mean())
                    res['std_accuracy'].append(cvs.std())
        time.sleep(1)
                   
    return res
                                        
            

In [14]:
params = {'max_depth': [x for x in range(1,50)], 'n_estimators': [5, 10, 50, 100], \
                                               'class_weight': ['balanced', None], 'criterion': ['entropy', 'gini']}

In [15]:
res = manual_grid_search(params)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49/49 [01:58<00:00,  2.42s/it]


In [16]:
res_my_grid = pd.DataFrame(res).sort_values(by='mean_accuracy', ignore_index=True, ascending=False)
res_my_grid

,n_estimators,max_depth,class_weight,criterion,mean_accuracy,std_accuracy
0,50,28,None,gini,0.904290,0.010961
1,100,31,None,gini,0.903547,0.014380
2,50,30,balanced,gini,0.902817,0.013554
3,50,34,balanced,gini,0.902809,0.013010
4,100,47,None,gini,0.902806,0.010460
...,...,...,...,...,...,...
779,5,1,None,entropy,0.353832,0.016467
780,5,2,balanced,entropy,0.353110,0.021165
781,5,2,balanced,gini,0.346419,0.029749
782,5,1,balanced,gini,0.283390,0.011062


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [17]:
predict = grid_rf.predict(X_test)

In [18]:
accuracy_score(y_test, predict)

0.9289940828402367